In [ ]:
import torch
import sys
sys.path.append('..')
from transformers import AutoModelForCausalLM, AutoTokenizer
import lre.models as models
import lre.functional as functional
import os

import json
import random
from lre.data import Relation, RelationSample, Sequence
import lre.metrics as metrics
import lre.functional as functional

device = 'cuda:0'

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b")
model = AutoModelForCausalLM.from_pretrained(
    "gemma_shards",
    torch_dtype=torch.bfloat16
)
model.to(device)
#tokenizer.pad_token = tokenizer.eos_token
mt = models.ModelAndTokenizer(model,tokenizer)

isinstance(mt, models.ModelAndTokenizer)

In [2]:
import llra.build as build
from importlib import reload
reload(functional)
reload(models)
reload(build)

<module 'llra.build' from '/home/exia/my-lre/data/../llra/build.py'>

In [ ]:
#I need to improve the Gemma prompting before I start working on the LRE.

start, end = 24, 41
APPROX_FOLDER = f'gemma_{start}_{end}_approx'
WEIGHT_NAME = f's_o_weight_{start}_{end}'
BIAS_NAME = f's_o_bias_{start}_{end}'
json_path = 'json/enckno/E06 [animal - youth].json'
file = open(json_path, 'r')
data = json.load(file)
file.close()

import llra.build as build
from baukit.baukit import parameter_names, get_parameter
import torch.nn as nn
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#OBTAIN LRE WEIGHTS
relations = os.listdir('gemma_12_41_approx')

def find_exact_match(filename, search_string):
    with open(filename, 'r') as file:
        for line in file:
            if search_string.lower() in line.strip().lower():
                return line.strip()
    print(f"Could not find {search_string}")
    return None

N_ICL = 8 
N_AVG = 8
N_TRIALS = 8
VIEW_SAMPLES = 5

for beta in range(1,20,2):
    for relation in relations:
        wdir = f"{APPROX_FOLDER}/{relation}"
        ignored_names = ['.ipynb_checkpoints', 'disappoint']
        j_samples = [x for x in os.listdir(wdir) if x not in ignored_names]
        j_samples = random.sample(j_samples, N_AVG)
        # print(f'using {N_AVG} samples out of {len(j_samples)} to build LRE')
        reg_weight = build.mean_weight_or_bias(wdir,
                                               WEIGHT_NAME, j_samples).half().to(device)
        reg_bias = build.mean_weight_or_bias(wdir,
                                             BIAS_NAME, j_samples).half().to(device)
        json_path = find_exact_match("json_paths.txt", relation)
        file = open(json_path, 'r')
        data = json.load(file)
        file.close()
        
        relation = Relation.from_dict(data)
        prompt_template = relation.prompt_templates[0]
        
        #ASSEMBLE PROMPTS AND OBJECT ANSWERS
        clozed_prompts = []
        clozed_answers = []
        #Do not use the training examples for ICL or testing purposes.
        test_samples = [x for x in relation.samples if x not in j_samples]
        random.shuffle(test_samples)
        for x in test_samples:
            test_samples_no_x = [t for t in test_samples if t != x]
            samples = [x] + random.sample(test_samples_no_x, N_ICL - 1)
            cloze_prompt = functional.make_prompt(
                template = prompt_template, 
                target = x,
                examples = samples
                )
            clozed_prompts.append(cloze_prompt)
            clozed_answers.append(x.object)
        
        outputs_lm = functional.predict_next_token(mt=mt, prompt=clozed_prompts)
        preds_lm =  [[x.token for x in xs] for xs in outputs_lm]
        recall_lm = metrics.recall(preds_lm, clozed_answers)
        reg_correct = 0
        no_bias_correct = 0
        first_token_correct = 0
        lm_correct = 0
        for i, sample, objs, prompt, preds in \
        zip(range(0,50), test_samples, clozed_answers, clozed_prompts, preds_lm):
            # if(i < VIEW_SAMPLES):
            #     print(f'{prompt} gemma: {preds[0]}')
            if (metrics.any_is_nontrivial_prefix(predictions=preds, targets=objs)):
                # # print(prompt)
                #uses the regular LRE
                reg_hs = build.get_hidden_state(mt, prompt, sample.subject, start)
                #reg_hs_end = build.get_hidden_state(mt, prompt, sample.subject, end)
                reg_hs = reg_hs.to(dtype=torch.float16)
                reg_hsweight = reg_hs.mm(reg_weight.t())
                reg_object_hs = reg_hsweight * beta + reg_bias
                reg_preds = build.get_object(mt, reg_object_hs)[0]
                
                no_bias_hs = reg_hsweight
                no_bias_preds = build.get_object(mt, no_bias_hs)[0]

                no_weight_hs = beta * reg_hs + reg_bias
                no_weight_preds = build.get_object(mt, no_weight_hs)[0]
                # init_subj_id = mt.tokenizer(sample.subject).input_ids[0]
                # init_subj_token = mt.tokenizer.convert_ids_to_tokens(init_subj_id)
                #print(init_subj_token)
                
                if(metrics.any_is_nontrivial_prefix(predictions=[reg_preds[0]], targets=objs)):
                    reg_correct += 1
    
                if(metrics.any_is_nontrivial_prefix(predictions=[no_bias_preds[0]], targets=objs)):
                    no_bias_correct += 1
            
                if(metrics.any_is_nontrivial_prefix(predictions=[no_weight_preds[0]], targets=objs)):
                    first_token_correct += 1
                    
                lm_correct += 1
    
                #if we want to compare specific samples from each relation..
                if(i < VIEW_SAMPLES):
                    #print(f'{sample.subject} {reg_preds[0:3]} {no_bias_preds[0:3]} {no_weight_preds[0:3]}')
                    pass
        print(f'{relation.name},{lm_correct}')
        #print(f'beta,{beta},{relation.name},reg_correct,{reg_correct},no_bias_correct,{no_bias_correct},lm_correct,{lm_correct}')

name - nationality,2
animal - youth,8
verb_Ving - 3pSg,8
noun+less_reg,5
verb+able_reg,8
UK_city - county,7
antonyms - binary,7
verb_inf - 3pSg,50
re+verb_reg,1
verb_inf - Ved,50
country - language,9
meronyms - part,1
verb_Ving - Ved,50
animal - shelter,8
hypernyms - misc,4
meronyms - substance,1
noun - plural_irreg,12
un+adj_reg,46
verb+ment_irreg,42
adj+ness_reg,3
over+adj_reg,4
verb+er_irreg,6
adj+ly_reg,50
name - occupation,0
synonyms - intensity,6
animal - sound,14
noun - plural_reg,50
Ving - verb_inf,7
male - female,1
verb_3pSg - Ved,14
meronyms - member,1
things - color,50
hyponyms - misc,6
adj - superlative,21
gemma_24_41_approx/verb+tion_irreg/examine/s_o_weight_24_41.pt does not exist, skipping
gemma_24_41_approx/verb+tion_irreg/examine/s_o_weight_24_41.pt does not exist, skipping
gemma_24_41_approx/verb+tion_irreg/examine/s_o_bias_24_41.pt does not exist, skipping
gemma_24_41_approx/verb+tion_irreg/examine/s_o_bias_24_41.pt does not exist, skipping
verb+tion_irreg,11
synonym

In [5]:
mt = models.ModelAndTokenizer(model,tokenizer)
from lre.models import ModelAndTokenizer
isinstance(mt, ModelAndTokenizer)

True

In [14]:
mt.model.lm_head(

Linear(in_features=3584, out_features=256000, bias=False)

In [66]:
import lre.lretyping as lretyping
from importlib import reload
reload(lretyping)
isinstance(mt.model,lretyping.Model)

True

In [12]:
import transformers
isinstance(mt.model, transformers.models.gemma2.modeling_gemma2.Gemma2ForCausalLM)

True